In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from beakerx.object import beakerx
#from dense import test_func

In [3]:
pd.DataFrame([[1,2,3],[1,2,3]])

,0,1,2
0,1,2,3
1,1,2,3


In [5]:
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(0)

In [6]:
np.random.seed(0)

In [7]:
x = pd.read_csv("Data/x_train.csv", index_col=0).values.reshape(-1, 28, 28)
y = pd.read_csv("Data/y_train.csv", index_col=0).values
x_test = pd.read_csv("Data/x_test.csv", index_col=0).values.reshape(-1, 28, 28)
y_test = pd.read_csv("Data/y_test.csv", index_col=0).values

In [8]:
x, y, x_test, y_test = map(torch.tensor, (x, y, x_test, y_test))
x = x.float().reshape(-1, 784)
y = y.float().argmax(1)
x_test = x_test.float().reshape(-1, 784)
y_test = y_test.float().argmax(1)

In [10]:
torch.manual_seed(0)
np.random.seed(0)
weights = torch.randn(784, 10) / np.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)
bs = 64
n, c = x.shape

In [11]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)
def model(xb):
    return (xb @ weights + bias)
def nll(input, target):
    return -input[range(target.shape[0]), target].mean()
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [12]:
xb = x[0:bs]
preds = model(xb)
print(preds[0], preds.shape)

tensor([-0.6534, -0.7182, -0.0832, -0.4927,  0.1882,  0.5409,  0.0754,  0.2682,
        -0.1369,  0.0101], grad_fn=<SelectBackward>) torch.Size([64, 10])


In [13]:
preds.dtype

torch.float32

In [15]:
loss_func = F.cross_entropy
yb = y[0:bs]
print(loss_func(preds, yb))

tensor(2.4617, grad_fn=<NllLossBackward>)


In [16]:
print(accuracy(preds, yb))

tensor(0.0469)


In [17]:
preds.shape, yb.shape

(torch.Size([64, 10]), torch.Size([64]))

In [18]:
lr = 0.5
epochs = 2
for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x[start_i:end_i]
        yb = y[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

In [19]:
accuracy(model(x_test), y_test)

tensor(0.9122)

In [20]:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        self.weights = nn.Parameter(torch.randn(784, 10) / np.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, xb):
        return xb @ self.weights + self.bias

In [21]:
model = Mnist_Logistic()
loss_func(model(x_test), y_test)

tensor(2.4008, grad_fn=<NllLossBackward>)

In [22]:
def fit():
    for epoch in range(epochs):
        for i in range((n - 1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x[start_i:end_i]
            yb = y[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

In [23]:
fit()

In [24]:
accuracy(model(x_test), y_test)

tensor(0.9122)

In [25]:
class Mnist_Logistic2(nn.Module):
    def __init__(self):
        torch.manual_seed(0)
        super().__init__()
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)

In [26]:
model = Mnist_Logistic()
print(loss_func(model(x_test), y_test))

tensor(2.4008, grad_fn=<NllLossBackward>)


In [27]:
fit()

print(loss_func(model(x_test), y_test))

tensor(0.3032, grad_fn=<NllLossBackward>)


In [28]:
accuracy(model(x_test), y_test)

tensor(0.9122)

In [29]:
def get_model():
    model = Mnist_Logistic2()
    return model, optim.SGD(model.parameters(), lr=lr)

In [30]:
model, opt = get_model()
print(loss_func(model(x_test), y_test))

tensor(2.3465, grad_fn=<NllLossBackward>)


In [31]:
for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x[start_i:end_i]
        yb = y[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(x_test), y_test))
print(accuracy(model(x_test), y_test))

tensor(0.3031, grad_fn=<NllLossBackward>)
tensor(0.9128)


In [32]:
train_ds = TensorDataset(x, y)

In [33]:
model, opt = get_model()
for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        xb, yb = train_ds[i * bs: i * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))
print(accuracy(model(x_test), y_test))

tensor(0.0533, grad_fn=<NllLossBackward>)
tensor(0.9128)


In [34]:
train_dl = DataLoader(train_ds, batch_size=bs)

In [35]:
model, opt = get_model()
for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))
print(accuracy(model(x_test), y_test))

tensor(0.0533, grad_fn=<NllLossBackward>)
tensor(0.9128)


In [36]:
valid_ds = TensorDataset(x_test, y_test)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

In [37]:
model, opt = get_model()
for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)
        

    print(epoch, valid_loss / len(valid_dl))

0 tensor(0.3098)
1 tensor(0.3007)


In [38]:
accuracy(model(x_test), y_test)

tensor(0.9128)

In [39]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [40]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(*[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl])
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

In [41]:
def get_data(x_train, x_valid, y_train, y_valid, bs):
    return (
        DataLoader(TensorDataset(x_train, y_train), batch_size=bs, shuffle=True),
        DataLoader(TensorDataset(x_valid, y_valid), batch_size=bs * 2),
    )

In [42]:
train_dl, valid_dl = get_data(x, x_test, y, y_test, bs)

In [43]:
model, opt = get_model()

In [44]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.3106646298646927
1 0.3264575546979904


In [45]:
loss_func = F.cross_entropy

In [46]:
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

In [47]:
model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=.9)

In [48]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.3831981011867523
1 0.3190640813946724


In [49]:
accuracy(model(x_test), y_test)

tensor(0.8999)